In [1]:
from google.colab import drive
drive.mount('/content/drive')

#Path to the folder
nifti_path = "/content/drive/MyDrive/Glioma_NIfTI_Dataset/UCSF-PDGM-v1.0-nifti"
csv_path = "/content/drive/MyDrive/Glioma_NIfTI_Dataset/UCSF-PDGM-metadata_v5.csv"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install nibabel
!pip install opencv-python
!pip install nilearn

In [3]:
import pandas as pd

df = pd.read_csv(csv_path)

# Filter rows with WHO CNS Grade 2, 3, or 4
df = df[df['WHO CNS Grade'].isin(['2', '3', '4'])]

# Keep only subject ID and grade, and clean column names
df = df[['ID', 'WHO CNS Grade']]
df['WHO CNS Grade'] = df['WHO CNS Grade'].astype(int)
df = df.rename(columns={'WHO CNS Grade': 'Grade'})

In [ ]:
import pandas as pd

df = pd.read_csv(csv_path)
print("Column names in CSV:")
print(list(df.columns))


In [ ]:
import os

nifti_root = '/content/drive/MyDrive/Glioma_NIfTI_Dataset/'  # or wherever your dataset is
sample_subjects = os.listdir(nifti_root)
print("Example subject folders/files:", sample_subjects[:5])

for subject_id in df['ID'].values[:5]:
    subject_path = os.path.join(nifti_root, subject_id)
    print(f"Checking: {subject_path} -> Exists: {os.path.exists(subject_path)}")


In [ ]:
df_full = pd.read_csv(csv_path)
print("Unique WHO Grades:", df_full['WHO CNS Grade'].unique())


In [ ]:
import os
import pandas as pd
import nibabel as nib
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#1. Paths
nifti_root = "/content/drive/MyDrive/Glioma_NIfTI_Dataset/PKG - UCSF-PDGM_NIfTI_for_BraTS2021/UCSF-PDGM-for-BraTS2021/UCSF-PDGM-nifti/"
csv_path = "/content/drive/MyDrive/Glioma_NIfTI_Dataset/UCSF-PDGM-metadata_v5.csv"

#2. Load metadata
df = pd.read_csv(csv_path)
print(f"✅ Metadata loaded. Shape: {df.shape}")

#3. Scan all FLAIR files
flair_map = {}
for root, dirs, files in os.walk(nifti_root):
    for file in files:
        if file.endswith("_FLAIR.nii.gz"):
            # Extract ID like 'UCSF-PDGM-0529' from filename
            subject_id = file.split("_FLAIR")[0]
            flair_map[subject_id] = os.path.join(root, file)

print(f"🧠 Found {len(flair_map)} FLAIR files.")

import re

def normalize_id(subject_id):
    match = re.search(r'UCSF-PDGM-(\d+)', subject_id)
    if match:
        return f"UCSF-PDGM-{int(match.group(1)):04d}"
    else:
        return None  # or subject_id if you want to keep unmatched ones

df["ID"] = df["ID"].apply(normalize_id)
df = df.dropna(subset=["ID"]) #Drops any rows where ID normalization failed

#4. Load images and labels
X = []
y = []
missing_ids = []

def load_nifti_image(path):
    img = nib.load(path).get_fdata()
    # Take central slice from 3D volume (axis 2), normalize, resize
    img_slice = img[:, :, img.shape[2] // 2]
    img_slice = cv2.resize(img_slice, (128, 128))
    img_slice = img_slice / np.max(img_slice)
    return img_slice

for _, row in df.iterrows():
    subject_id = row["ID"]
    grade = row["WHO CNS Grade"]

    if pd.isna(grade) or subject_id not in flair_map:
        missing_ids.append(subject_id)
        continue

    try:
        path = flair_map[subject_id]
        img_array = load_nifti_image(path)
        X.append(img_array)
        y.append(int(grade))
    except Exception as e:
        print(f"⚠️ Error loading {subject_id}: {e}")
        missing_ids.append(subject_id)

print(f"❌ Missing FLAIR for {len(missing_ids)} subjects: {missing_ids[:5]}")
print(f"✅ Loaded {len(X)} samples.")

# 5. Preprocess
if len(X) == 0:
    raise ValueError("❌ No images were loaded. Check paths or file structure.")

X = np.array(X).reshape(-1, 128, 128, 1)
y = to_categorical(np.array(y) - 2, num_classes=3)  # Grades 2/3/4 → 0/1/2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"✅ Data split: Train {len(X_train)}, Test {len(X_test)}")


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.utils import to_categorical

# ⚠️ Ensure y is properly shaped and encoded
y = np.array(y)
if len(y.shape) > 1:
    y = y.argmax(axis=-1)  # undo previous one-hot if already done

# 🧠 Encode WHO Grades (2, 3, 4) → (0, 1, 2)
if len(y.shape) == 1:
    y = y - 2  # encode labels 2,3,4 -> 0,1,2
    y = to_categorical(y, num_classes=3)


# 📦 Reshape image data
X = np.array(X).reshape(-1, 128, 128, 1)

# 📊 Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 🏗️ Define CNN model
model = Sequential([
    Input(shape=(128, 128, 1)),  # 👈 Input layer
    Conv2D(32, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Conv2D(64, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 3 output classes (Grade 2, 3, 4)
])

# ⚙️ Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🧪 Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=16,
    verbose=1
)


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# 🧹 Step 1: Ensure X is numeric and already 4D (with channel)
X = np.array(X, dtype=np.float32)
if X.ndim != 4 or X.shape[-1] != 1:
    raise ValueError(f"Expected X to have shape (n, 128, 128, 1), but got {X.shape}")

# 🎯 Step 2: Process labels
y = np.array(y)
print("Unique labels in y:", np.unique(y))

if np.any((y < 2) | (y > 4)):
    raise ValueError("Labels must be WHO Grades 2, 3, or 4 only.")

# Encode WHO Grades: (2, 3, 4) → (0, 1, 2)
y_encoded = y - 2

# ✅ Step 3: Train/test split
X_train, X_test, y_train_raw, y_test_raw = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 🔢 Step 4: One-hot encode labels
y_train = to_categorical(y_train_raw, num_classes=3)
y_test = to_categorical(y_test_raw, num_classes=3)

# 🏗️ Step 5: Define CNN model
model = Sequential([
    Input(shape=(128, 128, 1)),
    Conv2D(32, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Conv2D(64, kernel_size=3, activation='relu'),
    MaxPooling2D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # Output layer for 3 classes
])

# ⚙️ Step 6: Compile the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🚀 Step 7: Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=16,
    verbose=1
)

# 🧪 Step 8: Evaluate model on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Final Test Accuracy: {test_accuracy:.4f} | Test Loss: {test_loss:.4f}")


In [ ]:
import os

# ✅ Path where your .nii.gz files are stored
data_dir = "/content/drive/MyDrive/Glioma_NIfTI_Dataset/"

# Collect paths and labels
image_paths = []
labels = []

for file in os.listdir(data_dir):
    if file.endswith("_FLAIR.nii.gz"):
        patient_id = file.split("_")[0]  # e.g., UCSF-PDGM-0529
        label = id_to_grade.get(patient_id)
        if label in [2, 3, 4]:
            image_paths.append(os.path.join(data_dir, file))
            labels.append(label)

print(f"Total images found: {len(image_paths)}")
print(f"Sample label distribution: {np.unique(labels, return_counts=True)}")

import nibabel as nib
import cv2
import numpy as np

# Convert WHO Grades to 0, 1, 2 for classification
labels = [l - 2 for l in labels]  # 2→0, 3→1, 4→2

# Load MRI slices (center slice of FLAIR for simplicity)
def load_image(file_path):
    img = nib.load(file_path).get_fdata()
    center_slice = img.shape[2] // 2
    slice_2d = img[:, :, center_slice]
    resized = cv2.resize(slice_2d, (128, 128))
    return resized.astype(np.float32) / 255.0

X = np.array([load_image(p) for p in image_paths])
X = np.expand_dims(X, axis=-1)  # shape → (N, 128, 128, 1)
y = np.array(labels)

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

print("Total number of images:", len(X))
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Sample labels:", np.unique(y, return_counts=True))


# 📊 Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# One-hot encode labels
y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)


In [ ]:
import nibabel as nib
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
IMG_SIZE = 150
X = []
y = []

classes = ["Grade_2", "Grade_3", "Grade_4"]

for label, grade in enumerate(classes):
    folder_path = os.path.join(nifti_path, grade)
    for file in os.listdir(folder_path):
        if file.endswith(".nii") or file.endswith(".nii.gz"):
            img_path = os.path.join(folder_path, file)
            nii = nib.load(img_path)
            img_data = nii.get_fdata()

            mid_slice = img_data.shape[2] // 2
            img_slice = img_data[:, :, mid_slice]

            img_slice = cv2.resize(img_slice, (IMG_SIZE, IMG_SIZE))
            img_slice = img_slice / np.max(img_slice)  # Normalize

            X.append(img_slice)
            y.append(label)

X = np.array(X)
X = np.expand_dims(X, axis=-1)  # Add channel dimension
y = np.array(y)

# STEP 4: Train-Test Split and One-Hot Encoding
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

# STEP 5: Define the CNN Model
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),

    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# STEP 6: Train the Model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# STEP 7: Evaluate the Model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy:", test_acc)